To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",  # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",  # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",  # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",  # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",  # Gemma 2x faster!
]  # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-27b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.3.18: Fast Gemma2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.18 patched 46 layers with 46 QKV layers, 46 O layers and 46 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
# alpaca_prompt = """Below is an instruction that describes a response, paired with a context field that provides further context. Write a response that appropriately answers the request.

# ### Question:
# {}

# ### Context:
# {}

# ### Answer:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["question"]
#     inputs       = examples["context"]
#     outputs      = examples["answers"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return {"text": texts}


# from datasets import load_dataset
# dataset = load_dataset("SajjadAyoubi/persian_qa", split = "train")
# train_test_dataset = dataset.train_test_split(test_size=0.2) #splits the train split into train and test.

# train_dataset = train_test_dataset["train"]
# test_dataset = train_test_dataset["test"]
# train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

alpaca_prompt = """Below is an instruction that describes a response,
paired with a context field that provides further context.
Write a response that appropriately answers the request.

### Question:
{}

### Context:
{}

### Answer:
{}"""

# if "answers" in item and "text" in item["answers"] and len(item["answers"]["text"]) > 0:
#         reference = item["answers"]["text"][0]  # Assuming a list of answers
#       else:
#           reference = ""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answers"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        answer_text = output['text'][0] if output.get('text', []) else ''
        text = alpaca_prompt.format(instruction, input, answer_text) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}
pass

from datasets import load_dataset
dataset_train = load_dataset("SajjadAyoubi/persian_qa", split = "train")
dataset_test = load_dataset("SajjadAyoubi/persian_qa", split = "validation")
dataset = dataset_train.map(formatting_prompts_func, batched = True,)

In [ ]:
print(dataset[40])

{'id': 51, 'title': 'کتاب', 'context': 'کتاب (وام\u200cواژه از زبان عربی، جمع: کُتُب) مجموعه\u200cای از صفحاتِ نوشته\u200cشده، مصوّر، چاپ\u200cشده یا صفحات خالی (صفحه سفید و نانوشته)؛ ساخته\u200cشده است. از لوح\u200cهای گِلی، در منطقهٔ بین\u200cالنهرین برای نوشتن استفاده می\u200cشد. استفاده از این روش در ۲۵۰۰ سال قبل از میلاد، توسط تمدن سومر ابداع شد و به تکامل رسید. اما با گذشت زمان، بابلی\u200cها و آشوری\u200cها نیز همین روش را به\u200cکار گرفتند. این لوح\u200cها با استفاده از خاکِ رس و آب ساخته می\u200cشدند و قبل از خشک شدن، با استفاده از ابزارهای نوک\u200cتیز بر روی آن\u200cها نوشته می\u200cشد. نوشته\u200cهای بیشتر این لوح\u200cها، بیشتر مرتبط با امر بازرگانی، اداری و حکومتی بود.[۳] کدکس شکل امروزی کتاب است؛ یعنی صفحات پوست، پاپیروس و غیره که تا خورده و از یک محل به یکدیگر دوخته شده باشند. پیدایش کدکس به قرنِ دوم میلادی بر می\u200cگردد که مسیحیان از آن برای نگارش متون مذهبی استفاده می\u200cکردند. این فرم از کتابْ به\u200cدلیل مزیت\u200cهایی که داشت، از قرن چهارم میلادی رواج پیدا کر

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 90,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "qw",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.684 GB of memory reserved.


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,008 | Num Epochs = 1 | Total steps = 90
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 114,180,096/27,000,000,000 (0.42% trained)
AUTOTUNE bmm(256x456x128, 256x128x456)
  bmm 0.2253 ms 100.0% 
  triton_bmm_14 0.4219 ms 53.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_10 0.4690 ms 48.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_5 0.4844 ms 46.5% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  trito

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.957700
2,2.031700
3,2.103200
4,1.961700
5,1.891400
6,1.848300
7,1.769400
8,1.606900
9,1.679700
10,1.662000


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [9]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "fdsfsdfdsf؟", # instruction
        "ddsdds؟", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

AUTOTUNE bmm(16x55x256, 16x256x55)
  triton_bmm_153 0.0133 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_166 0.0143 ms 92.9% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_155 0.0154 ms 86.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_159 0.0154 ms 86.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=8
  triton_bmm_158 0.0164 ms 81.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_161 0.0164

['<bos>Below is an instruction that describes a response,\npaired with a context field that provides further context.\nWrite a response that appropriately answers the request.\n\n### Question:\nfdsfsdfdsf؟\n\n### Context:\nddsdds؟\n\n### Answer:\n<eos>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "تعداد درختان سیب کشف شده؟", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a response,
paired with a context field that provides further context.
Write a response that appropriately answers the request.

### Question:
تعداد درختان سیب کشف شده؟

### Context:


### Answer:
تعداد درختان سیب کشف شده: 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [7]:
from huggingface_hub import login #writing to huggingface
login(token="")

In [8]:
model.push_to_hub("regd/gemma227b90step2e-4")
trainer.create_model_card("regd/gemma227b90step2e-4")
tokenizer.push_to_hub("regd/gemma227b90step2e-4")

README.md:   0%|          | 0.00/575 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

Saved model to https://huggingface.co/regd/gemma227b90step2e-4


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

In [1]:
from huggingface_hub import login #reading from huggingface
login(token="")

In [2]:
%pip install -U bitsandbytes --quiet
%pip install datasets --quiet


In [3]:
%pip install tqdm --quiet
%pip install unsloth --quiet


In [4]:
%pip install bert-score --quiet

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # Import the AutoModel class
model_name = "regd/gemma227b90step2e-4"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name) # Now AutoModel should be defined


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# dataset_test
def format_prompt(question, context=None):
         if context:
             prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
         else:
             prompt = f"Question: {question}\n\nAnswer:"
         return prompt

In [11]:
def generate_answer(model, tokenizer, prompt, max_new_tokens=20, max_seq_length=2048):
         inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
         with torch.no_grad():
             outputs = model.generate(
                 **inputs,
                 max_new_tokens=max_new_tokens,
                 temperature=0.7, # Adjust as needed
                 top_p=0.9,      # Adjust as needed
                 do_sample=False,
                 max_length = 20,
                 eos_token_id=tokenizer.eos_token_id,
                 use_cache=False,
                #  num_beams=5,
                #  early_stopping=True
             )
         answer = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
         return answer.strip()



In [12]:
def evaluate_f1_token(prediction, reference):
          pred_tokens = prediction.lower().split()
          ref_tokens = reference.lower().split()
          common = set(pred_tokens) & set(ref_tokens)
          if len(pred_tokens) == 0 or len(ref_tokens) == 0:
              return 0.0
          precision = len(common) / len(pred_tokens)
          recall = len(common) / len(ref_tokens)
          if precision + recall == 0:
              return 0.0
          f1 = 2 * precision * recall / (precision + recall)
          return f1

In [13]:
def evaluate_exact_match(prediction, reference):
    """
    Calculates the Exact Match (EM) score between a prediction and a reference.

    Args:
        prediction (str): The predicted answer.
        reference (str): The reference answer.

    Returns:
        float: 1.0 if the prediction exactly matches the reference, 0.0 otherwise.
    """
    if prediction.strip().lower() == reference.strip().lower():
        return 1.0
    else:
        return 0.0

In [15]:
from bert_score import score

def evaluate_bert_score(prediction, reference, lang="en", model_type=None, num_layers=None, verbose=False):
    """
    Calculates BERTScore for a single prediction and reference.

    Args:
        prediction (str): The predicted sentence.
        reference (str): The reference sentence.
        lang (str): Language of the sentences (e.g., "en", "es").
        model_type (str, optional): BERT model to use (e.g., "bert-base-uncased"). If None, it will be automatically inferred from lang.
        num_layers (int, optional): Number of BERT layers to use.
        verbose (bool, optional): Print progress.

    Returns:
        float: The F1-score from BERTScore, or 0.0 if there is an error.
    """
    try:
        P, R, F1 = score([prediction], [reference], lang=lang, model_type=model_type, num_layers=num_layers, verbose=verbose)
        return F1.item()  # Return the F1 score as a float
    except Exception as e:
        print(f"Error calculating BERTScore: {e}")
        return 0.0

# Example usage (replace your existing evaluate_f1_token calls):
# ... (your existing code) ...

# Replace calls to evaluate_f1_token with evaluate_bert_score
# f1_score = evaluate_f1_token(prediction, reference)
# f1_score = evaluate_bert_score(prediction, reference)

# ... (rest of your code) ...

In [16]:
import torch
from tqdm import tqdm
from datasets import load_dataset
dataset_test = load_dataset("SajjadAyoubi/persian_qa", split = "validation")
dataset_test = dataset_test.select(range(100))
results = []
for item in tqdm(dataset_test, desc="Evaluating"):
      question = item["question"]
      if "answers" in item and "text" in item["answers"] and len(item["answers"]["text"]) > 0:
        reference = item["answers"]["text"][0]  # Assuming a list of answers
      else:
          reference = ""
      context = item.get("context", None)
      prompt = format_prompt(question, context)
      prediction = generate_answer(model, tokenizer, prompt)
      f1_score = evaluate_f1_token(prediction, reference)
      em_score = evaluate_exact_match(prediction, reference)
      bert_score = evaluate_bert_score(prediction, reference)
      results.append({
        "question": question,
        "reference": reference,
        "prediction": prediction,
        "f1_score": f1_score,
        "exact_match": em_score,
        "bert_score": bert_score
      })
results



Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_

[{'question': 'پایتخت اسپانیا کجاست؟',
  'reference': 'مادرید',
  'prediction': '',
  'f1_score': 0.0,
  'exact_match': 0.0,
  'bert_score': 0.0},
 {'question': 'بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال اروپا است؟',
  'reference': 'فیفا',
  'prediction': '',
  'f1_score': 0.0,
  'exact_match': 0.0,
  'bert_score': 0.0},
 {'question': 'رئال مادرید چند بار در لیگ قهرمانان اروپا به عنوان قهرمانی رسیده؟',
  'reference': '۱۳',
  'prediction': '',
  'f1_score': 0.0,
  'exact_match': 0.0,
  'bert_score': 0.0},
 {'question': 'معنی واژه رئال به اسپانیایی چیست؟',
  'reference': 'سلطنتی',
  'prediction': '',
  'f1_score': 0.0,
  'exact_match': 0.0,
  'bert_score': 0.0},
 {'question': 'تیم رئال مادرید برای کجاست؟',
  'reference': 'مادرید، پایتخت اسپانیا',
  'prediction': '',
  'f1_score': 0.0,
  'exact_match': 0.0,
  'bert_score': 0.0},
 {'question': 'لقب باشگاه رئال مادرید از کجا میاد؟',
  'reference': 'شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد',
  'prediction': '',
  'f1_score': 0.0,


In [17]:
import numpy as np
avg_f1 = np.mean([result["f1_score"] for result in results])
avg_em = np.mean([result["exact_match"] for result in results])
avg_bert = np.mean([result["bert_score"] for result in results])
print(f"Average F1 Score: {avg_f1:.4f}")
print(f"Average Exact Match: {avg_em:.4f}")
print(f"Average BERT Score: {avg_bert:.4f}")

Average F1 Score: 0.0000
Average Exact Match: 0.3000
Average BERT Score: 0.0000


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
    [
        alpaca_prompt.format(
            "What is a famous tall tower in Paris?",  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
tokenizer.batch_decode(outputs)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )